<a href="https://colab.research.google.com/github/bel4life/Diabetes---Pima-Indians-Study/blob/master/Diabetes_Pima_Indians_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import pandas
import numpy
import sklearn
import keras
import pandas as pd
import numpy as np

In [38]:
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes (2).csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))

In [0]:
names = ['n_Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
        'BMI', 'PedigreeFunction', 'Age', 'Outcome']

In [41]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [42]:
df[df['Glucose'] == 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [43]:
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    df.dropna(inplace=True)
    
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [44]:
dataset = df.values
print(dataset.shape)

(392, 9)


In [0]:
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [46]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 8)
(392,)
[0 1 1 1 1]


**Normalise the Data**

In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [48]:
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


**Importing SkLearn & Keras packages for Model Analysis**

In [0]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [50]:
def create_model():

    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_88 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_89 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_90 (Dense)             (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


**Do a grid search for the optimal batch size and number of epochs**

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam


In [0]:
seed = 6
np.random.seed(seed)

In [0]:
def create_model():
    
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn = create_model, verbose = 1)

In [0]:
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

In [0]:
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [57]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/10
261/261 [==============================] - 2s 8ms/step - loss: 0.6294 - acc: 0.6897
Epoch 2/10
261/261 [==============================] - 0s 164us/step - loss: 0.4809 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 165us/step - loss: 0.4408 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 162us/step - loss: 0.4257 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 245us/step - loss: 0.4166 - acc: 0.8008
Epoch 6/10
261/261 [==============================] - 0s 171us/step - loss: 0.4068 - acc: 0.8123
Epoch 7/10
261/261 [==============================] - 0s 173us/step - loss: 0.4041 - acc: 0.8084
Epoch 8/10
261/261 [==============================] - 0s 165us/step - loss: 0.3991 - acc: 0.8123
Epoch 9/10
261/261 [==============================] - 0s 169us/step - loss: 0.3981 - acc: 0.8084
Epoch 10/10
131/131 [==============================] - 1s 7ms/step
[CV] ............ batch_size=10, epochs=10, score=0.756, total

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 2s 9ms/step - loss: 0.6381 - acc: 0.6475
Epoch 2/10
261/261 [==============================] - 0s 172us/step - loss: 0.5025 - acc: 0.7241
Epoch 3/10
261/261 [==============================] - 0s 165us/step - loss: 0.4573 - acc: 0.7931
Epoch 4/10
261/261 [==============================] - 0s 163us/step - loss: 0.4439 - acc: 0.7816
Epoch 5/10
261/261 [==============================] - 0s 187us/step - loss: 0.4383 - acc: 0.7969
Epoch 6/10
261/261 [==============================] - 0s 171us/step - loss: 0.4332 - acc: 0.8008
Epoch 7/10
261/261 [==============================] - 0s 180us/step - loss: 0.4259 - acc: 0.8046
Epoch 8/10
261/261 [==============================] - 0s 188us/step - loss: 0.4153 - acc: 0.8046
Epoch 9/10
261/261 [==============================] - 0s 201us/step - loss: 0.4033 - acc: 0.8238
Epoch 10/10
131/131 [==============================] - 1s 7ms/step
[CV] ............ batch_size=10, epochs=10, score=0.763, total

...

Epoch 1/100
262/262 [==============================] - 3s 10ms/step - loss: 0.6448 - acc: 0.6336
Epoch 2/100
262/262 [==============================] - 0s 172us/step - loss: 0.5360 - acc: 0.6527
Epoch 3/100
262/262 [==============================] - 0s 170us/step - loss: 0.5115 - acc: 0.6527
Epoch 4/100
262/262 [==============================] - 0s 181us/step - loss: 0.4982 - acc: 0.7290
Epoch 5/100
262/262 [==============================] - 0s 176us/step - loss: 0.4922 - acc: 0.7786
Epoch 6/100
262/262 [==============================] - 0s 174us/step - loss: 0.4871 - acc: 0.7672
Epoch 7/100
262/262 [==============================] - 0s 179us/step - loss: 0.4818 - acc: 0.7672
Epoch 8/100
262/262 [==============================] - 0s 180us/step - loss: 0.4779 - acc: 0.7710
Epoch 9/100
262/262 [==============================] - 0s 200us/step - loss: 0.4704 - acc: 0.7863
Epoch 10/100
262/262 [==============================] - 0s 193us/step - loss: 0.4649 - acc: 0.7672
Epoch 11/100
262/262

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   58.5s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 3s 10ms/step - loss: 0.6717 - acc: 0.6590
Epoch 2/10
261/261 [==============================] - 0s 109us/step - loss: 0.5594 - acc: 0.7280
Epoch 3/10
261/261 [==============================] - 0s 113us/step - loss: 0.4423 - acc: 0.8008
Epoch 4/10
261/261 [==============================] - 0s 111us/step - loss: 0.4100 - acc: 0.8008
Epoch 5/10
261/261 [==============================] - 0s 108us/step - loss: 0.3948 - acc: 0.8123
Epoch 6/10
261/261 [==============================] - 0s 107us/step - loss: 0.3822 - acc: 0.8199
Epoch 7/10
261/261 [==============================] - 0s 109us/step - loss: 0.3765 - acc: 0.8238
Epoch 8/10
261/261 [==============================] - 0s 108us/step - loss: 0.3806 - acc: 0.8123
Epoch 9/10
261/261 [==============================] - 0s 125us/step - loss: 0.3790 - acc: 0.8199
Epoch 10/10
131/131 [==============================] - 1s 8ms/step
[CV] ............ batch_size=20, epochs=10, score=0.718, tota

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.9min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/100
392/392 [==============================] - 4s 10ms/step - loss: 0.6057 - acc: 0.6633
Epoch 2/100
392/392 [==============================] - 0s 214us/step - loss: 0.4937 - acc: 0.6684
Epoch 3/100
392/392 [==============================] - 0s 218us/step - loss: 0.4784 - acc: 0.7219
Epoch 4/100
392/392 [==============================] - 0s 214us/step - loss: 0.4649 - acc: 0.7857
Epoch 5/100
392/392 [==============================] - 0s 213us/step - loss: 0.4614 - acc: 0.7934
Epoch 6/100
392/392 [==============================] - 0s 210us/step - loss: 0.4503 - acc: 0.7883
Epoch 7/100
392/392 [==============================] - 0s 214us/step - loss: 0.4541 - acc: 0.7959
Epoch 8/100
392/392 [==============================] - 0s 220us/step - loss: 0.4454 - acc: 0.8010
Epoch 9/100
392/392 [==============================] - 0s 222us/step - loss: 0.4345 - acc: 0.7959
Epoch 10/100
392/392 [==============================] - 0s 217us/step - loss: 0.4251 - acc: 0.7959
Epoch 11/100
392/392

In [64]:

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Best: 0.7857142817609164, using {'batch_size': 10, 'epochs': 100}
0.7755101976954207 (0.02288290529576492) with: {'batch_size': 10, 'epochs': 10}
0.7576530627450164 (0.018440091885642364) with: {'batch_size': 10, 'epochs': 50}
0.7857142817609164 (0.04809360162688935) with: {'batch_size': 10, 'epochs': 100}
0.7678571469625648 (0.039973842332125704) with: {'batch_size': 20, 'epochs': 10}
0.7729591862583647 (0.027465942869593005) with: {'batch_size': 20, 'epochs': 50}
0.7780612188638473 (0.05888433193632155) with: {'batch_size': 20, 'epochs': 100}
0.7653061171271363 (0.04098916578779024) with: {'batch_size': 40, 'epochs': 10}
0.7704081663063594 (0.043052406128042585) with: {'batch_size': 40, 'epochs': 50}
0.7704081495805662 (0.046457990547614986) with: {'batch_size': 40, 'epochs': 100}


In [0]:
from keras.layers import Dropout

**Do a grid search to optimize kernel initialization and activation functions using necessary packages**

In [0]:
seed = 6
np.random.seed(seed)

In [0]:

def create_model(activation, init):
    
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model


In [0]:
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

In [0]:
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

In [0]:
param_grid = dict(activation = activation, init = init)


In [71]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV] .... activation=softmax, init=uniform, score=0.756, total=  10.0s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.0s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.756, total=  10.0s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   20.0s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.4min remaining:    0.0s


[CV] ....... activation=softmax, init=zero, score=0.700, total=  11.6s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.6min remaining:    0.0s


[CV] ....... activation=relu, init=uniform, score=0.725, total=  12.0s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.756, total=  12.2s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.823, total=  12.9s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.718, total=  12.4s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.763, total=  11.7s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.823, total=  11.7s
[CV] activation=relu, init=zero ......................................
[CV] .......... activation=relu, init=zero, score=0.611, total=  11.9s
[CV] activation=relu, init=zero ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  7.5min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [72]:
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Best: 0.7933673531729348, using {'activation': 'tanh', 'init': 'normal'}
0.7755102136609505 (0.028087641954935023) with: {'activation': 'softmax', 'init': 'uniform'}
0.7295918416003792 (0.08860773018104312) with: {'activation': 'softmax', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'softmax', 'init': 'zero'}
0.7678571475707755 (0.04085046948442003) with: {'activation': 'relu', 'init': 'uniform'}
0.7678571427050902 (0.0431682294462949) with: {'activation': 'relu', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'relu', 'init': 'zero'}
0.7831632721484924 (0.02879958859349298) with: {'activation': 'tanh', 'init': 'uniform'}
0.7933673531729348 (0.032371719468446684) with: {'activation': 'tanh', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'tanh', 'init': 'zero'}
0.7933673531729348 (0.037313655933021314) with: {'activation': 'linear', 'init': 'uniform'}
0.7908163291155076 (0.03370616593

**Do a grid search to find the optimal number of neurons in each hidden layer using necessary packages**

In [0]:
seed = 6
np.random.seed(seed)

In [0]:

def create_model(neuron1, neuron2):
    
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

In [0]:
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

In [0]:
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

In [0]:
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

In [78]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................
[CV] ................ neuron1=4, neuron2=2, score=0.771, total=  15.3s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.3s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.763, total=  14.9s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   30.2s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.1min remaining:    0.0s


[CV] ................ neuron1=4, neuron2=8, score=0.831, total=  16.1s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.3min remaining:    0.0s


[CV] ................ neuron1=8, neuron2=2, score=0.763, total=  16.4s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.763, total=  16.5s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.838, total=  16.6s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.763, total=  16.9s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.763, total=  17.4s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.838, total=  17.6s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ................ neuron1=8, neuron2=8, score=0.763, total=  17.5s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  7.7min finished


In [79]:
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Best: 0.7908163351976142, using {'neuron1': 16, 'neuron2': 2}
0.785714290123813 (0.02650267677503863) with: {'neuron1': 4, 'neuron2': 2}
0.7882653126607135 (0.019356087898682556) with: {'neuron1': 4, 'neuron2': 4}
0.785714290123813 (0.03173682706863349) with: {'neuron1': 4, 'neuron2': 8}
0.7882653141812402 (0.03535836258888925) with: {'neuron1': 8, 'neuron2': 2}
0.7882653111401869 (0.03535836473101593) with: {'neuron1': 8, 'neuron2': 4}
0.7857142931648663 (0.031736824926506764) with: {'neuron1': 8, 'neuron2': 8}
0.7908163351976142 (0.03897990025127174) with: {'neuron1': 16, 'neuron2': 2}
0.7908163306360342 (0.03370616199600475) with: {'neuron1': 16, 'neuron2': 4}
0.7857142931648663 (0.031736824926506764) with: {'neuron1': 16, 'neuron2': 8}


In [80]:
y_pred = grid.predict(X_standardized)
print(y_pred.shape)

(392, 1)


In [81]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


**Classification Report**

In [82]:
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7806122448979592
              precision    recall  f1-score   support

           0       0.81      0.89      0.84       262
           1       0.71      0.57      0.63       130

    accuracy                           0.78       392
   macro avg       0.76      0.73      0.74       392
weighted avg       0.77      0.78      0.77       392

